In [2]:
from IPython.core.display import display, HTML 
display(HTML("<style>.container { width:70% !important; }</style>")) 

# データ読み込み等

In [56]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
import os
os.chdir("C:\\Users\\takumi\\Desktop\\L-pro_repair\\L-pro\\201609-201708")

In [57]:
df_up70 = pd.read_csv("df_up70.csv")
df_up70['受付日']=pd.to_datetime(df_up70['受付日'],format='%Y%m%d')
del df_up70['Unnamed: 0']
df_up70.index = df_up70['受付日']
df_up70.sort_index(inplace=True)

今回は地域を受付NO登場回数が上位70%に絞っているので行数は去年よりも少なくなる

# 欠損値処理

In [58]:
#欠損値処理
df_dropna = df_up70.dropna(subset=['受付NO','部品品番','拠点','受付日','大現象名','品番グループ','修理依頼内容','一発修理FLG'])
#df_dropna['受付NO'].value_counts()
#重複行処理
df_dropna = df_dropna.drop_duplicates()

In [59]:
#受付NOが2つ以上
g2 = df_dropna[(df_dropna.duplicated('受付NO'))]
#受付NOが1つだけ
g3 = df_dropna[(~df_dropna.duplicated('受付NO'))]

#g2の中で'大現象名','品番グループ','修理依頼内容'が1パターンではないもの
#g2[~g2.duplicated(['大現象名','品番グループ','修理依頼内容'])]
#'大現象名','品番グループ','修理依頼内容'が1パターンであるもの
g4 = g2[g2.duplicated(['大現象名','品番グループ','修理依頼内容'])]
df = pd.concat([g3,g4]).sort_index()

# 学習データの絞り込み

In [ ]:
df = df[('2016-09-01' <= df['受付日']) & (df['受付日'] <= '2017-08-31')]

# dfの中で頻出部品選出

# Mecabで形態素解析

## mecab-ipadic-neologd のエントリを生成する際の正規化処理
https://github.com/neologd/mecab-ipadic-neologd/wiki/Regexp

In [61]:
from __future__ import unicode_literals
import re
import unicodedata

def unicode_normalize(cls, s):
    pt = re.compile('([{}]+)'.format(cls))

# unicodedata.normalizeのNFKC（Normalization Form Compatibility Composition）で
# 半角カタカナ、全角英数、ローマ数字・丸数字、異体字などなどを正規化
    def norm(c):
        return unicodedata.normalize('NFKC', c) if pt.match(c) else c

    s = ''.join(norm(x) for x in re.split(pt, s))
    s = re.sub('－', '-', s)
    return s

def remove_extra_spaces(s):
    s = re.sub('[ 　]+', ' ', s)
    blocks = ''.join(('\u4E00-\u9FFF',  # CJK UNIFIED IDEOGRAPHS
                      '\u3040-\u309F',  # HIRAGANA
                      '\u30A0-\u30FF',  # KATAKANA
                      '\u3000-\u303F',  # CJK SYMBOLS AND PUNCTUATION
                      '\uFF00-\uFFEF'   # HALFWIDTH AND FULLWIDTH FORMS
                      ))
    basic_latin = '\u0000-\u007F'

    def remove_space_between(cls1, cls2, s):
        p = re.compile('([{}]) ([{}])'.format(cls1, cls2))
        while p.search(s):
            s = p.sub(r'\1\2', s)
        return s

    s = remove_space_between(blocks, blocks, s)
    s = remove_space_between(blocks, basic_latin, s)
    s = remove_space_between(basic_latin, blocks, s)
    return s

def normalize_neologd(s):
    s = s.strip()
    s = unicode_normalize('０-９Ａ-Ｚａ-ｚ｡-ﾟ', s)

    def maketrans(f, t):
        return {ord(x): ord(y) for x, y in zip(f, t)}

    s = re.sub('[˗֊‐‑‒–⁃⁻₋−]+', '-', s)  # normalize hyphens
    s = re.sub('[﹣－ｰ—―─━ー]+', 'ー', s)  # normalize choonpus
    s = re.sub('[~∼∾〜〰～]', '', s)  # remove tildes
    s = s.translate(
        maketrans('!"#$%&\'()*+,-./:;<=>?@[¥]^_`{|}~｡､･｢｣',
              '！”＃＄％＆’（）＊＋，－．／：；＜＝＞？＠［￥］＾＿｀｛｜｝〜。、・「」'))

    s = remove_extra_spaces(s)
    s = unicode_normalize('！”＃＄％＆’（）＊＋，－．／：；＜＞？＠［￥］＾＿｀｛｜｝〜', s)  # keep ＝,・,「,」
    s = re.sub('[’]', '\'', s)
    s = re.sub('[”]', '"', s)
    return s

In [62]:
# 大文字を小文字に変換
def lower_text(text):
    return text.lower()

# 数字を０で置換
def normalize_number(text):
    # 連続した数字を0で置換
    replaced_text = re.sub(r'\d+', '0', text)
    return replaced_text

In [63]:
df["修理依頼内容"] = df["修理依頼内容"].apply(normalize_neologd)
df["修理依頼内容"] = df["修理依頼内容"].apply(lower_text)
df["修理依頼内容"] = df["修理依頼内容"].apply(normalize_number)

# 特定文字列の置換

In [64]:
df['修理依頼内容'] = df['修理依頼内容'].str.replace('シャワ-','シャワー')
df['修理依頼内容'] = df['修理依頼内容'].str.replace('センサ-','センサー')
df['修理依頼内容'] = df['修理依頼内容'].str.replace('レバ-','レバー')

# '\dF'　1Fの階の意味と、部品等のTFを区別するため
df['修理依頼内容'] = df['修理依頼内容'].replace('\df', '階',regex=True)

# 単語の分割

In [65]:
import MeCab
def wakati_by_mecab(text):
    tagger = MeCab.Tagger('')
    tagger.parse('') 
# parseToNode()を使うと形態素の詳細情報が得らる
# parseToNode()は先頭のノード（形態素情報）を返し、surfaceで表層形、featureで形態素情報を取得
# featureは , で区切られているのでsplit()などで分割して必要な情報を抽出
    node = tagger.parseToNode(text)
    word_list = []
    while node:
        pos = node.feature.split(",")[0]
        if pos in ["名詞", "動詞", "形容詞","副詞"]:   # 対象とする品詞
            word = node.surface
            word_list.append(word)
        node = node.next
    return word_list

In [67]:
df["修理依頼内容"] = df["修理依頼内容"].apply(wakati_by_mecab)

### ストップワードの除去

In [69]:
import os
import urllib.request

def download_stopwords(path):
    url = 'http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt'
    if os.path.exists(path):
        print('File already exists.')
    else:
        print('Downloading...')
        # Download the file from `url` and save it locally under `file_name`:
        urllib.request.urlretrieve(url, path)

def create_stopwords(file_path):     
    stop_words = [w.replace('\n','') for w in open(path, "r",encoding="utf-8_sig") if len(w) > 0]
    return stop_words    

path = "stop_words.txt"
download_stopwords(path)
stop_words = create_stopwords(path)
stop_words.remove('水')
stop_words.remove('春')
stop_words.remove('夏')
stop_words.remove('秋')
stop_words.remove('冬')
stop_words.append("0")

File already exists.


In [70]:
print(stop_words)

['あそこ', 'あたり', 'あちら', 'あっち', 'あと', 'あな', 'あなた', 'あれ', 'いくつ', 'いつ', 'いま', 'いや', 'いろいろ', 'うち', 'おおまか', 'おまえ', 'おれ', 'がい', 'かく', 'かたち', 'かやの', 'から', 'がら', 'きた', 'くせ', 'ここ', 'こっち', 'こと', 'ごと', 'こちら', 'ごっちゃ', 'これ', 'これら', 'ごろ', 'さまざま', 'さらい', 'さん', 'しかた', 'しよう', 'すか', 'ずつ', 'すね', 'すべて', 'ぜんぶ', 'そう', 'そこ', 'そちら', 'そっち', 'そで', 'それ', 'それぞれ', 'それなり', 'たくさん', 'たち', 'たび', 'ため', 'だめ', 'ちゃ', 'ちゃん', 'てん', 'とおり', 'とき', 'どこ', 'どこか', 'ところ', 'どちら', 'どっか', 'どっち', 'どれ', 'なか', 'なかば', 'なに', 'など', 'なん', 'はじめ', 'はず', 'はるか', 'ひと', 'ひとつ', 'ふく', 'ぶり', 'べつ', 'へん', 'ぺん', 'ほう', 'ほか', 'まさ', 'まし', 'まとも', 'まま', 'みたい', 'みつ', 'みなさん', 'みんな', 'もと', 'もの', 'もん', 'やつ', 'よう', 'よそ', 'わけ', 'わたし', '', 'ハイ', '', '', '上', '中', '下', '字', '', '', '年', '月', '日', '時', '分', '秒', '週', '火', '木', '金', '土', '国', '都', '道', '府', '県', '市', '区', '町', '村', '', '', '各', '第', '方', '何', '的', '度', '文', '者', '性', '体', '人', '他', '今', '部', '課', '係', '外', '類', '達', '気', '室', '口', '誰', '用', '界', '会', '首', '男', '女', '別', '話', '私', '屋', '店', '家', '場', '等'

In [71]:
def remove_stopwords(words):
    words = [word for word in words if word not in stop_words]
    return words

In [72]:
df["修理依頼内容"] = df["修理依頼内容"].apply(remove_stopwords)

# ひらがな、カタカナ1文字のものを削除

In [73]:
def remove_onecharacter(a):
    remove_list = []
    for ch in a:
        if len(ch) == 1:
            name = unicodedata.name(ch)
            if "HIRAGANA" in name or "KATAKANA" in name:
                remove_list.append(ch)
    
    words = [word for word in a if word not in remove_list]
    return words

In [74]:
def remove_onecharacter(a):
    remove_list = []
    for ch in a:
        if len(ch) == 1:
            name = unicodedata.name(ch)
            if "HIRAGANA" in name or "KATAKANA" in name:
                remove_list.append(ch)
    
    words = [word for word in a if word not in remove_list]
    return words

In [75]:
df["修理依頼内容"] = df["修理依頼内容"].apply(remove_onecharacter)

In [76]:
df.to_csv("wakati_201609-201708.csv")

# 似た意味を持つ表記の統一

In [77]:
def replace_1(a):
    replace1 = [s.replace("蓋","ふた") for s in a]
    return replace1
def replace_2(a):
    replace2 = [s.replace("フタ","ふた") for s in a]
    return replace2
def replace_3(a):
    replace3 = [s.replace("尻","おしり") for s in a]
    return replace3
def replace_4(a):
    replace4 = [s.replace("漏れる","漏れ") for s in a]
    return replace4
def replace_5(a):
    replace5 = [s.replace("0階","階") for s in a]
    return replace5
def replace_6(a):
    replace6 = [s.replace("落ちる","落ち") for s in a]
    return replace6
def replace_7(a):
    replace7 = [s.replace("流す","流れ") for s in a]
    return replace7
def replace_8(a):
    replace8 = [s.replace("流せ","流れ") for s in a]
    return replace8
def replace_9(a):
    replace9 = [s.replace("流れる","流れ") for s in a]
    return replace9
def replace_10(a):
    replace10 = [s.replace("流し","流れ") for s in a]
    return replace10
def replace_11(a):
    replace11 = [s.replace("裏側","裏") for s in a]
    return replace11
def replace_12(a):
    replace12 = [s.replace("閉まっ","閉まる") for s in a]
    return replace12
def replace_13(a):
    replace13 = [s.replace("閉じる","閉まる") for s in a]
    return replace13

In [78]:
df["修理依頼内容"] = df["修理依頼内容"].apply(replace_1)
df["修理依頼内容"] = df["修理依頼内容"].apply(replace_2)
df["修理依頼内容"] = df["修理依頼内容"].apply(replace_3)
df["修理依頼内容"] = df["修理依頼内容"].apply(replace_4)
df["修理依頼内容"] = df["修理依頼内容"].apply(replace_5)
df["修理依頼内容"] = df["修理依頼内容"].apply(replace_6)
df["修理依頼内容"] = df["修理依頼内容"].apply(replace_7)
df["修理依頼内容"] = df["修理依頼内容"].apply(replace_8)
df["修理依頼内容"] = df["修理依頼内容"].apply(replace_9)
df["修理依頼内容"] = df["修理依頼内容"].apply(replace_10)
df["修理依頼内容"] = df["修理依頼内容"].apply(replace_11)
df["修理依頼内容"] = df["修理依頼内容"].apply(replace_12)
df["修理依頼内容"] = df["修理依頼内容"].apply(replace_13)

# 頻出部品300のデータセット抽出  
# 頻出単語250を求める

In [79]:
#受付NOを除去する前のdfのコピーを取っておく
df2 = df.copy()

In [80]:
#受付NOの重複を除去(多くの部品が含まれる受付は修理依頼内容がかぶるために重複を除かなければならない)
df = df.drop_duplicates('受付NO')

どの期間の頻出部品を扱うかを決定

In [81]:
# 頻出部品３００
freq_list = df["部品品番"].value_counts()
freq_list_300 = freq_list[:300]
df300 = df[(df["部品品番"].isin(freq_list_300.index))]
# df300.to_csv("df300.csv")

buhin_freq = pd.DataFrame(freq_list)
buhin_300 = buhin_freq.head(300)
buhin_300.to_csv('buhin300_201609-201708.csv')

In [82]:
df_freq300 = df300

In [31]:
# #リスト内包表記に書き換え(計算時間を速くするため)
# y = [list(df_freq300["修理依頼内容"].values[i] for i in range(len(df_freq300)))]

In [83]:
y = []
for i in range(len(df_freq300)):
# for i in range(3):
    w = list(df_freq300["修理依頼内容"][i])
    y.append(w)

In [84]:
#     """2重のリストをフラットにする関数"""
def flatten(nested_list):
    return [e for inner_list in nested_list for e in inner_list]

y = flatten(y)

In [85]:
import collections
pd.set_option('display.max_rows', 250)
count = collections.Counter(y)
df_count = pd.DataFrame.from_dict(count, orient='index').reset_index()
df_count.columns=['単語','出現回数']
df_count2 = df_count.sort_values('出現回数',ascending=False)
df_250_new=df_count2.head(250).reset_index().drop('index',axis=1)

In [86]:
df_250 = df_250_new
df_250_new.to_csv("freq250_201609-201708.csv")
df2.to_csv("wakati_201609-201708.csv")

# 現象名・品番グループ整理

In [ ]:
#大現象名・品番グループについてダミー変数化
df_up70_dummy = pd.get_dummies(df2,columns=['大現象名','品番グループ'])#ダミー変数化

# 単語の特徴ベクトル作成

In [89]:
#頻出単語ファイル読み込み(途中から始めるときはここでデータを読み込む)
df_250 = pd.read_csv('freq250_201609-201708.csv').drop("Unnamed: 0",axis=1)
df_250

,単語,出現回数
0,水,56053
1,漏れ,37996
2,便器,23276
3,タンク,21061
4,階,19021
5,ノズル,12466
6,床,11866
7,流れ,11566
8,出,10925
9,便座,10557


In [90]:
#単語の列を作る
for i in range(250):
    df_up70_dummy[df_250.iloc[i]["単語"]] = 0

In [93]:
#この後の作業でかなり時間がかかるので念のためcsvに書き出し
df_up70_dummy.to_csv('df_up70_dummy_201609-201708.csv')
test = df_up70_dummy.copy()

# ここからAnacondaプロンプト

In [41]:
#anaconda 
#頻出単語ファイル読み込み(途中から始めるときはここでデータを読み込む)
df_250 = pd.read_csv('freq250_201609-201708.csv').drop("Unnamed: 0",axis=1)
df_up70_dummy =pd.read_csv("df_up70_dummy_201609-201708.csv")
test = df_up70_dummy.copy()

修理依頼内容に頻出単語が含まれていたら1を当てる  

# 高速化

for文の中のlocやilocはかなり時間がかかるのでnumpyをベースにしたコードに書き換え

In [ ]:
test_syuri = test["修理依頼内容"].values 
#test_syuri = test["出現回数"].values   #anaconda用に変えた
df_250_tango = df_250["単語"].values  
#df_250_tango = df_250["出現回数"].values      #
for i in tqdm(range(len(df_up70_dummy))):
    a = test_syuri[i]
    for k in a:
        for p in range(len(df_250)):
            if k == df_250_tango[p]:
                test[df_250_tango[p]][i] = 1  

In [115]:
df_250.iloc[1]["単語"]
df_250["単語"].values[1]

'漏れ'

In [38]:
test.to_csv("201609-201708ex_data.csv")

In [42]:
#exdata = pd.read_csv("ex_data.csv",encoding='cp932',index_col=[0])
exdata = pd.read_csv("201609-201708ex_data.csv",index_col=[0])

In [44]:
dataset=test.drop(["拠点","受付日.1","品番中グループ","商品品番","商品品目","修理依頼内容","社内メッセージ","得意先メッセージ","一発修理FLG"],axis=1)
dataset = dataset.drop(["いる","する","なる","てる"],axis=1)

# テストデータ作成

テストデータにはない特徴量がテストデータには含まれているので
その分を除いて学習させなければならない

In [ ]:
#テストデータ読み込み
df_test = pd.read_csv('lixil_test.csv',encoding='cp932').dropna(subset=['受付NO','部品品番','拠点','受付日','大現象名','品番グループ','修理依頼内容','一発修理FLG'])
#重複行処理
df_test = df_test.drop_duplicates()

In [98]:
#頻出単語・頻出部品読み込み
#df_250 = pd.read_csv('freq250_201509_201609.csv',encoding='cp932').drop("Unnamed: 0",axis=1)
df_250 = pd.read_csv('freq250_201609-201708.csv').drop("Unnamed: 0",axis=1)
#buhin300 = pd.read_csv("buhin300_201509_201609.csv",encoding='cp932').drop("部品品番",axis=1)
buhin300 = pd.read_csv("buhin300_201609-201708.csv").drop("部品品番",axis=1)

In [100]:
df_test["修理依頼内容"] = df_test["修理依頼内容"].apply(normalize_neologd)
df_test["修理依頼内容"] = df_test["修理依頼内容"].apply(lower_text)
df_test["修理依頼内容"] = df_test["修理依頼内容"].apply(normalize_number)
df_test['修理依頼内容'] = df_test['修理依頼内容'].str.replace('シャワ-','シャワー')
df_test['修理依頼内容'] = df_test['修理依頼内容'].str.replace('センサ-','センサー')
df_test['修理依頼内容'] = df_test['修理依頼内容'].str.replace('レバ-','レバー')
# '\dF'　1Fの階の意味と、部品等のTFを区別するため
df_test['修理依頼内容'] = df_test['修理依頼内容'].replace('\df', '階',regex=True)
df_test["修理依頼内容"] = df_test["修理依頼内容"].apply(wakati_by_mecab)
df_test["修理依頼内容"] = df_test["修理依頼内容"].apply(remove_onecharacter)
df_test["修理依頼内容"] = df_test["修理依頼内容"].apply(replace_1)
df_test["修理依頼内容"] = df_test["修理依頼内容"].apply(replace_2)
df_test["修理依頼内容"] = df_test["修理依頼内容"].apply(replace_3)
df_test["修理依頼内容"] = df_test["修理依頼内容"].apply(replace_4)
df_test["修理依頼内容"] = df_test["修理依頼内容"].apply(replace_5)
df_test["修理依頼内容"] = df_test["修理依頼内容"].apply(replace_6)
df_test["修理依頼内容"] = df_test["修理依頼内容"].apply(replace_7)
df_test["修理依頼内容"] = df_test["修理依頼内容"].apply(replace_8)
df_test["修理依頼内容"] = df_test["修理依頼内容"].apply(replace_9)
df_test["修理依頼内容"] = df_test["修理依頼内容"].apply(replace_10)
df_test["修理依頼内容"] = df_test["修理依頼内容"].apply(replace_11)
df_test["修理依頼内容"] = df_test["修理依頼内容"].apply(replace_12)
df_test["修理依頼内容"] = df_test["修理依頼内容"].apply(replace_13)

In [101]:
df_test_cl = df_test.copy()
# df_test_cl['部品品番'] = df_test_cl['部品品番'].str.replace(',','')

#大現象名・品番グループについてダミー変数化
df_test_dummy = pd.get_dummies(df_test_cl,columns=['大現象名','品番グループ'])#ダミー変数化

pd.set_option('display.max_columns', 50)

In [102]:
for i in range(250):
    df_test_dummy[df_250.iloc[i]["単語"]] = 0    

# 高速化

In [103]:
df_test_dummy_syuri = df_test_dummy["修理依頼内容"].values
df_250_tango = df_250["単語"].values
for i in tqdm(range(len(df_test_dummy))):
# for i in tqdm(range(10)):
#     print(df_up70_dummy.iloc[i]["修理依頼内容"])
    a = df_test_dummy_syuri[i]
    for k in a:
        for p in range(len(df_250)):
            if k == df_250_tango[p]:
                df_test_dummy[df_250_tango[p]][i] = 1 

100%|██████████████████████████████████████████████████████████████████████████████| 1191/1191 [00:24<00:00, 48.20it/s]


In [104]:
dataset_test =df_test_dummy.drop(["拠点","受付日","品番中グループ","商品品番","商品品目","修理依頼内容","社内メッセージ","得意先メッセージ","一発修理FLG"],axis=1)
dataset_test = dataset_test.drop(["いる","する","なる","てる"],axis=1)

In [106]:
dataset_test.to_csv('dataset_test_201709.csv')

In [107]:
len(dataset_test)

1191